In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive')#change dir

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras.applications import imagenet_utils
from skimage import data, color, feature

import os
from tqdm import tqdm_notebook as tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
from keras.applications.resnet import ResNet101
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet import ResNet152
from keras.applications.inception_v3 import InceptionV3
from keras.applications import MobileNet
from keras.applications import Xception # TensorFlow ONLY
from keras.applications import VGG16

In [ ]:
df_train = pd.read_excel('/content/drive/My Drive/Labels.xlsx')


In [ ]:
df_train.head()

,ImageName,Level
0,fall-01-cam0-rgb-001.png,1
1,fall-01-cam0-rgb-002.png,1
2,fall-01-cam0-rgb-003.png,1
3,fall-01-cam0-rgb-004.png,1
4,fall-01-cam0-rgb-005.png,1


In [ ]:
targets_series = pd.Series(df_train['Level'])
one_hot = pd.get_dummies(targets_series, sparse = True)

In [ ]:
one_hot_labels = np.asarray(one_hot)

In [ ]:
im_size1 = 128
im_size2 = 128

In [ ]:
x_train = []
y_train = []
x_test = []

In [ ]:
# i = 0 
# #for folder in range(9):
# for f, breed in tqdm(df_train.values):
#   if type(cv2.imread('IMAGES/{}'.format(f)))==type(None):
#     continue 
#   else: 
#     img = cv2.imread('IMAGES/{}'.format(f)) 
#     label = one_hot_labels[i] 
#     x_train.append(cv2.resize(img, (im_size1, im_size2))) 
#     y_train.append(label) 
#     i += 1 
# np.save('x_trainuURF',x_train) 
# np.save('y_trainURF',y_train) 
# print('Done')

In [ ]:
x_train = np.load('x_trainuURF.npy')
y_train = np.load('y_trainURF.npy')

In [ ]:
y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255.

In [ ]:
print(x_train_raw.shape)
print(y_train_raw.shape)

(14161, 128, 128, 3)
(14161, 4)


In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.1, random_state=1)

In [ ]:
num_class = y_train_raw.shape[1]

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.layers import AveragePooling2D
# from keras.layers import AveragePooling
#Optimized ResNet-50

In [ ]:
base_model = ResNet50(weights = None, include_top=False, input_shape=(im_size1, im_size2, 3))

# Add a new top layer
x = base_model.output
x = AveragePooling2D(pool_size=(4, 4))(x)
x = Flatten()(x)
x = Dropout(0.4)(x)
#x = Dense(1024, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)
#predictions = Dense(num_class, activation='sigmoid')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
#for layer in base_model.layers:
#    layer.trainable = False

# model.compile(loss='categorical_crossentropy', 
#               optimizer='RMSprop', 
#               metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adagrad', 
#               metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adam', 
#               metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', 
              optimizer='SGD', 
              metrics=['accuracy'])


callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', verbose=1)]
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [ ]:
model.fit(X_train, Y_train, epochs=5, validation_data=(X_valid, Y_valid), verbose=1,shuffle=True)

Epoch 1/5
399/399 [==============================] - 47s 119ms/step - loss: 1.1689 - accuracy: 0.4425 - val_loss: 1.1457 - val_accuracy: 0.5095
Epoch 2/5
399/399 [==============================] - 47s 118ms/step - loss: 0.9431 - accuracy: 0.5862 - val_loss: 1.1209 - val_accuracy: 0.4855
Epoch 3/5
399/399 [==============================] - 48s 120ms/step - loss: 0.5862 - accuracy: 0.7680 - val_loss: 1.3960 - val_accuracy: 0.5928
Epoch 4/5
399/399 [==============================] - 48s 121ms/step - loss: 0.3704 - accuracy: 0.8581 - val_loss: 0.4260 - val_accuracy: 0.8179
Epoch 5/5
399/399 [==============================] - 49s 122ms/step - loss: 0.2814 - accuracy: 0.8957 - val_loss: 0.2923 - val_accuracy: 0.8899


In [ ]:
X_train_features = []
X_valid_features = []

feature_network = Model(base_model.input, model.get_layer('flatten').output)

X_train_features = feature_network.predict(X_train)  # Assuming you have your images in x
X_valid_features = feature_network.predict(X_valid)  # Assuming you have your images in x

print(X_train_features.shape)
print(X_valid_features.shape)
print(Y_train.shape)
print(Y_valid.shape)

(12744, 2048)
(1417, 2048)
(12744, 4)
(1417, 4)


In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import time

# Hyper parameters
# C penalty parameter of error term. Smaller values -> stronger regularization.
param_grid = {'C': [1e-1, 1e0], 'max_iter': [500, 1000]}

# Create model and fit to training data. 
# Do grid search CV to find the best hyperparameters
start_time = time.time()
svm_orig = svm.LinearSVC(max_iter=1000, dual=False)
svm_orig = GridSearchCV(svm_orig, param_grid)
Y_Train_Array = np.argmax(Y_train, axis=1)
print(Y_Train_Array.shape)

svm_orig.fit(X=X_train_features, y=Y_Train_Array)
print("--- %s seconds ---" % (time.time() - start_time))

# Print model with chosen hyperparameters
print(svm_orig)

# Predict on test data
svm_predict_orig = svm_orig.predict(X_valid_features)

# Get accuracy
svm_acc_orig = (svm_predict_orig == np.argmax(Y_valid, axis=1)).mean()
print(svm_acc_orig)

(12744,)
--- 522.0850281715393 seconds ---
GridSearchCV(cv=None, error_score=nan,
             estimator=LinearSVC(C=1.0, class_weight=None, dual=False,
                                 fit_intercept=True, intercept_scaling=1,
                                 loss='squared_hinge', max_iter=1000,
                                 multi_class='ovr', penalty='l2',
                                 random_state=None, tol=0.0001, verbose=0),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 1.0], 'max_iter': [500, 1000]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)
0.9781227946365562
